Project: /mediapipe/_project.yaml
Book: /mediapipe/_book.yaml

<link rel="stylesheet" href="/mediapipe/site.css">

# Hand gesture recognition model customization guide

<table align="left" class="buttons">
  <td>
    <a href="https://colab.research.google.com/github/googlesamples/mediapipe/blob/main/examples/customization/gesture_recognizer.ipynb" target="_blank">
      <img src="https://developers.google.com/static/mediapipe/solutions/customization/colab-logo-32px_1920.png" alt="Colab logo"> Run in Colab
    </a>
  </td>

  <td>
    <a href="https://github.com/googlesamples/mediapipe/blob/main/examples/customization/gesture_recognizer.ipynb" target="_blank">
      <img src="https://developers.google.com/static/mediapipe/solutions/customization/github-logo-32px_1920.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
</table>

In [ ]:
#@title License information
# Copyright 2023 The MediaPipe Authors.
# Licensed under the Apache License, Version 2.0 (the "License");
#
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

The MediaPipe Model Maker package is a low-code solution for customizing on-device machine learning (ML) Models.

This notebook shows the end-to-end process of customizing a gesture recognizer model for recognizing some common hand gestures in the [HaGRID](https://www.kaggle.com/datasets/innominate817/hagrid-sample-30k-384p) dataset.

## Prerequisites

Install the MediaPipe Model Maker package.

In [1]:
!pip3 install --upgrade pip
!pip3 install mediapipe-model-maker

  Using cached pip-23.3.1-py3-none-any.whl (2.1 MB)
  Attempting uninstall: pip
    Found existing installation: pip 22.0.2
    Uninstalling pip-22.0.2:
      Successfully uninstalled pip-22.0.2
ERROR: Could not install packages due to an OSError: [Errno 13] Permission denied: 'top_level.txt'
Check the permissions.

  Using cached mediapipe_model_maker-0.2.1.3-py3-none-any.whl.metadata (1.6 kB)
  Using cached tensorflow_addons-0.22.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached tensorflow_datasets-4.9.3-py3-none-any.whl.metadata (9.3 kB)
  Using cached tensorflow_hub-0.15.0-py2.py3-none-any.whl.metadata (1.3 kB)
  Using cached tf_models_official-2.15.0-py2.py3-none-any.whl.metadata (1.4 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 321.6 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 1.6 MB/s eta 0:00:00a 0:00:010m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 432.6 kB/s eta 0

Import the required libraries.

In [2]:
#from google.colab import files
import os
import tensorflow as tf
assert tf.__version__.startswith('2')

from mediapipe_model_maker import gesture_recognizer

import matplotlib.pyplot as plt

2023-11-27 23:29:12.992631: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-27 23:29:12.994714: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-27 23:29:13.025083: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-27 23:29:13.025115: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-27 23:29:13.026025: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

## Simple End-to-End Example

This end-to-end example uses Model Maker to customize a model for on-device gesture recognition.

### Get the dataset

The dataset for gesture recognition in model maker requires the following format: `<dataset_path>/<label_name>/<img_name>.*`. In addition, one of the label names (`label_names`) must be `none`. The `none` label represents any gesture that isn't classified as one of the other gestures.

This example uses a rock paper scissors dataset sample which is downloaded from GCS.

In [3]:
# !wget https://storage.googleapis.com/mediapipe-tasks/gesture_recognizer/rps_data_sample.zip
# !unzip rps_data_sample.zip
dataset_path = "SigNN Character Database"

Verify the rock paper scissors dataset by printing the labels. There should be 4 gesture labels, with one of them being the `none` gesture.

In [ ]:
print(dataset_path)
labels = []
for i in os.listdir(dataset_path):
  if os.path.isdir(os.path.join(dataset_path, i)):
    labels.append(i)
print(labels)

To better understand the dataset, plot a couple of example images for each gesture.

In [ ]:
NUM_EXAMPLES = 5

for label in labels:
  label_dir = os.path.join(dataset_path, label)
  example_filenames = os.listdir(label_dir)[:NUM_EXAMPLES]
  fig, axs = plt.subplots(1, NUM_EXAMPLES, figsize=(10,2))
  for i in range(NUM_EXAMPLES):
    axs[i].imshow(plt.imread(os.path.join(label_dir, example_filenames[i])))
    axs[i].get_xaxis().set_visible(False)
    axs[i].get_yaxis().set_visible(False)
  fig.suptitle(f'Showing {NUM_EXAMPLES} examples for {label}')

plt.show()

### Run the example
The workflow consists of 4 steps which have been separated into their own code blocks.

**Load the dataset**

Load the dataset located at `dataset_path` by using the `Dataset.from_folder` method. When loading the dataset, run the pre-packaged hand detection model from MediaPipe Hands to detect the hand landmarks from the images. Any images without detected hands are ommitted from the dataset. The resulting dataset will contain the extracted hand landmark positions from each image, rather than images themselves.

The `HandDataPreprocessingParams` class contains two configurable options for the data loading process:
* `shuffle`: A boolean controlling whether to shuffle the dataset. Defaults to true.
* `min_detection_confidence`: A float between 0 and 1 controlling the confidence threshold for hand detection.

Split the dataset: 80% for training, 10% for validation, and 10% for testing.

In [4]:
data = gesture_recognizer.Dataset.from_folder(
    dirname=dataset_path,
    hparams=gesture_recognizer.HandDataPreprocessingParams()
)
train_data, rest_data = data.split(0.8)
validation_data, test_data = rest_data.split(0.5)

Using existing files at /tmp/model_maker/gesture_recognizer/palm_detection_full.tflite
Using existing files at /tmp/model_maker/gesture_recognizer/hand_landmark_full.tflite
INFO:tensorflow:Loading image /home/aadel/Desktop/awd/Project /SigNN Character Database/B/412.jpg
INFO:tensorflow:Loading image /home/aadel/Desktop/awd/Project /SigNN Character Database/A/294.jpg
INFO:tensorflow:Loading image /home/aadel/Desktop/awd/Project /SigNN Character Database/H/50.jpg
INFO:tensorflow:Loading image /home/aadel/Desktop/awd/Project /SigNN Character Database/K/226.jpg
INFO:tensorflow:Loading image /home/aadel/Desktop/awd/Project /SigNN Character Database/U/128.jpg
INFO:tensorflow:Loading image /home/aadel/Desktop/awd/Project /SigNN Character Database/L/133.jpg
INFO:tensorflow:Loading image /home/aadel/Desktop/awd/Project /SigNN Character Database/T/105.jpg


I0000 00:00:1701109767.587619   24892 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1701109767.590349   25340 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 Mesa 23.0.4-0ubuntu1~22.04.1), renderer: Mesa Intel(R) Xe Graphics (TGL GT2)
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


INFO:tensorflow:Loading image /home/aadel/Desktop/awd/Project /SigNN Character Database/G/317.jpg
INFO:tensorflow:Loading image /home/aadel/Desktop/awd/Project /SigNN Character Database/F/392.jpg
INFO:tensorflow:Loading image /home/aadel/Desktop/awd/Project /SigNN Character Database/Q/135.jpg
INFO:tensorflow:Loading image /home/aadel/Desktop/awd/Project /SigNN Character Database/T/260.jpg
INFO:tensorflow:Loading image /home/aadel/Desktop/awd/Project /SigNN Character Database/H/85.jpg
INFO:tensorflow:Loading image /home/aadel/Desktop/awd/Project /SigNN Character Database/T/122.jpg
INFO:tensorflow:Loading image /home/aadel/Desktop/awd/Project /SigNN Character Database/B/175.jpg
INFO:tensorflow:Loading image /home/aadel/Desktop/awd/Project /SigNN Character Database/A/437.jpg
INFO:tensorflow:Loading image /home/aadel/Desktop/awd/Project /SigNN Character Database/F/128.jpg
INFO:tensorflow:Loading image /home/aadel/Desktop/awd/Project /SigNN Character Database/A/168.jpg
INFO:tensorflow:Loadi

2023-11-27 23:33:38.379357: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-27 23:33:38.379719: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Using existing files at /tmp/model_maker/gesture_recognizer/gesture_embedder
INFO:tensorflow:Load valid hands with size: 6994, num_label: 22, labels: None,A,B,C,D,E,F,G,H,I,K,L,M,N,O,P,Q,R,S,T,U,V.


INFO:tensorflow:Load valid hands with size: 6994, num_label: 22, labels: None,A,B,C,D,E,F,G,H,I,K,L,M,N,O,P,Q,R,S,T,U,V.


**Train the model**

Train the custom gesture recognizer by using the create method and passing in the training data, validation data, model options, and hyperparameters. For more information on model options and hyperparameters, see the [Hyperparameters](#hyperparameters) section below.

In [5]:
hparams = gesture_recognizer.HParams(export_dir="exported_model")
options = gesture_recognizer.GestureRecognizerOptions(hparams=hparams)
model = gesture_recognizer.GestureRecognizer.create(
    train_data=train_data,
    validation_data=validation_data,
    options=options
)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hand_embedding (InputLayer  [(None, 128)]             0         
 )                                                               
                                                                 
 batch_normalization (Batch  (None, 128)               512       
 Normalization)                                                  
                                                                 
 re_lu (ReLU)                (None, 128)               0         
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 custom_gesture_recognizer_  (None, 22)                2838      
 out (Dense)                                                     
                                                             

INFO:tensorflow:Training the models...


Epoch 1/10
2797/2797 [==============================] - 5s 2ms/step - loss: 1.2942 - categorical_accuracy: 0.5243 - val_loss: 0.2648 - val_categorical_accuracy: 0.8298 - lr: 0.0010
Epoch 2/10
2797/2797 [==============================] - 5s 2ms/step - loss: 0.7726 - categorical_accuracy: 0.6941 - val_loss: 0.2104 - val_categorical_accuracy: 0.8684 - lr: 9.9000e-04
Epoch 3/10
2797/2797 [==============================] - 5s 2ms/step - loss: 0.6916 - categorical_accuracy: 0.7292 - val_loss: 0.1863 - val_categorical_accuracy: 0.9056 - lr: 9.8010e-04
Epoch 4/10
2797/2797 [==============================] - 5s 2ms/step - loss: 0.6433 - categorical_accuracy: 0.7492 - val_loss: 0.1821 - val_categorical_accuracy: 0.8941 - lr: 9.7030e-04
Epoch 5/10
2797/2797 [==============================] - 5s 2ms/step - loss: 0.6158 - categorical_accuracy: 0.7614 - val_loss: 0.1816 - val_categorical_accuracy: 0.8827 - lr: 9.6060e-04
Epoch 6/10
2797/2797 [==============================] - 7s 2ms/step - loss: 0.6

**Evaluate the model performance**

After training the model, evaluate it on a test dataset and print the loss and accuracy metrics.

In [6]:
loss, acc = model.evaluate(test_data, batch_size=1)
print(f"Test loss:{loss}, Test accuracy:{acc}")

700/700 [==============================] - 2s 1ms/step - loss: 0.1647 - categorical_accuracy: 0.9086
Test loss:0.16473248600959778, Test accuracy:0.9085714221000671


**Export to Tensorflow Lite Model**

After creating the model, convert and export it to a Tensorflow Lite model format for later use on an on-device application. The export also includes model metadata, which includes the label file.

In [7]:
model.export_model()
!ls exported_model

Using existing files at /tmp/model_maker/gesture_recognizer/palm_detection_full.tflite
Using existing files at /tmp/model_maker/gesture_recognizer/hand_landmark_full.tflite
INFO:tensorflow:Assets written to: /tmp/tmpe96vl1z_/saved_model/assets


INFO:tensorflow:Assets written to: /tmp/tmpe96vl1z_/saved_model/assets


best_model_weights.data-00000-of-00001	epoch_models		 metadata.json
best_model_weights.index		gesture_recognizer.task
checkpoint				logs


2023-11-27 23:35:22.112479: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2023-11-27 23:35:22.112512: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2023-11-27 23:35:22.112843: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpe96vl1z_/saved_model
2023-11-27 23:35:22.113391: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2023-11-27 23:35:22.113403: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmpe96vl1z_/saved_model
2023-11-27 23:35:22.114596: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
2023-11-27 23:35:22.115080: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2023-11-27 23:35:22.133815: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: 

In [ ]:
files.download('exported_model/gesture_recognizer.task')

## Run the model on-device

To use the TFLite model for on-device usage through MediaPipe Tasks, refer to the Gesture Recognizer [overview page](https://developers.google.com/mediapipe/solutions/vision/gesture_recognizer).

## Hyperparameters {:#hyperparameters}


You can further customize the model using the `GestureRecognizerOptions` class, which has two optional parameters for `ModelOptions` and `HParams`. Use the `ModelOptions` class to customize parameters related to the model itself, and the `HParams` class to customize other parameters related to training and saving the model.

`ModelOptions` has one customizable parameter that affects accuracy:
* `dropout_rate`: The fraction of the input units to drop. Used in dropout layer. Defaults to 0.05.
* `layer_widths`: A list of hidden layer widths for the gesture model. Each element in the list will create a new hidden layer with the specified width. The hidden layers are separated with BatchNorm, Dropout, and ReLU. Defaults to an empty list(no hidden layers).

`HParams` has the following list of customizable parameters which affect model accuracy:
* `learning_rate`: The learning rate to use for gradient descent training. Defaults to 0.001.
* `batch_size`: Batch size for training. Defaults to 2.
* `epochs`: Number of training iterations over the dataset. Defaults to 10.
* `steps_per_epoch`: An optional integer that indicates the number of training steps per epoch. If not set, the training pipeline calculates the default steps per epoch as the training dataset size divided by batch size.
* `shuffle`: True if the dataset is shuffled before training. Defaults to False.
* `lr_decay`: Learning rate decay to use for gradient descent training. Defaults to 0.99.
* `gamma`: Gamma parameter for focal loss. Defaults to 2

Additional `HParams` parameter that does not affect model accuracy:
* `export_dir`: The location of the model checkpoint files and exported model files.

For example, the following trains a new model with the dropout_rate of 0.2 and learning rate of 0.003.

In [ ]:
hparams = gesture_recognizer.HParams(learning_rate=0.003, export_dir="exported_model_2")
model_options = gesture_recognizer.ModelOptions(dropout_rate=0.2)
options = gesture_recognizer.GestureRecognizerOptions(model_options=model_options, hparams=hparams)
model_2 = gesture_recognizer.GestureRecognizer.create(
    train_data=train_data,
    validation_data=validation_data,
    options=options
)

Evaluate the newly trained model.

In [ ]:
loss, accuracy = model_2.evaluate(test_data)
print(f"Test loss:{loss}, Test accuracy:{accuracy}")